examples from https://cognitus.fr/mllib-p2-feature-transformation/

In [1]:
from pyspark.sql import SparkSession

In [2]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("hockey").getOrCreate()

In [11]:
file_loc = 'gs://wac-buck/colors.csv'

In [12]:
df = spark.read.format('csv').option('inferSchema',
                                    True).option('header',True).load(file_loc)

In [13]:
display(df)

DataFrame[id: int, color: string]

In [14]:
df.show()

+---+------+
| id| color|
+---+------+
|  1|   red|
|  2|  blue|
|  3|orange|
|  4| white|
|  5|   red|
|  6|orange|
|  7|   red|
|  8| white|
|  9|   red|
+---+------+



In [15]:
df.columns

['id', 'color']

# StringIndexer

In [16]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="color", outputCol="color_indexed")

In [17]:
indexer_model = indexer.fit(df)

In [18]:
indexed_data= indexer_model.transform(df)

In [19]:
indexed_data.show()

+---+------+-------------+
| id| color|color_indexed|
+---+------+-------------+
|  1|   red|          0.0|
|  2|  blue|          3.0|
|  3|orange|          2.0|
|  4| white|          1.0|
|  5|   red|          0.0|
|  6|orange|          2.0|
|  7|   red|          0.0|
|  8| white|          1.0|
|  9|   red|          0.0|
+---+------+-------------+



# OneHotEncoderEstimator
note that color is a nominal var meaning the order doesn't matter. This is why it's best to create dummies so that the model does not infer more significance to larger values.

In [20]:
from pyspark.ml.feature import OneHotEncoderEstimator
ohe = OneHotEncoderEstimator(inputCols=["color_indexed"], outputCols=["color_ohe"])

In [21]:
ohe_model = ohe.fit(indexed_data)


In [22]:
encoded_data = ohe_model.transform(indexed_data)
encoded_data.show()

+---+------+-------------+-------------+
| id| color|color_indexed|    color_ohe|
+---+------+-------------+-------------+
|  1|   red|          0.0|(3,[0],[1.0])|
|  2|  blue|          3.0|    (3,[],[])|
|  3|orange|          2.0|(3,[2],[1.0])|
|  4| white|          1.0|(3,[1],[1.0])|
|  5|   red|          0.0|(3,[0],[1.0])|
|  6|orange|          2.0|(3,[2],[1.0])|
|  7|   red|          0.0|(3,[0],[1.0])|
|  8| white|          1.0|(3,[1],[1.0])|
|  9|   red|          0.0|(3,[0],[1.0])|
+---+------+-------------+-------------+



# Scaling and normalizing

In [24]:
file_loc = 'gs://wac-buck/wine.data'
df = spark.read.csv(file_loc,header=False,
                    inferSchema=True)

In [25]:
df.show()

+---+-----+----+----+----+---+----+----+----+----+----+----+----+----+
|_c0|  _c1| _c2| _c3| _c4|_c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|
+---+-----+----+----+----+---+----+----+----+----+----+----+----+----+
|  1|14.23|1.71|2.43|15.6|127| 2.8|3.06|0.28|2.29|5.64|1.04|3.92|1065|
|  1| 13.2|1.78|2.14|11.2|100|2.65|2.76|0.26|1.28|4.38|1.05| 3.4|1050|
|  1|13.16|2.36|2.67|18.6|101| 2.8|3.24| 0.3|2.81|5.68|1.03|3.17|1185|
|  1|14.37|1.95| 2.5|16.8|113|3.85|3.49|0.24|2.18| 7.8|0.86|3.45|1480|
|  1|13.24|2.59|2.87|21.0|118| 2.8|2.69|0.39|1.82|4.32|1.04|2.93| 735|
|  1| 14.2|1.76|2.45|15.2|112|3.27|3.39|0.34|1.97|6.75|1.05|2.85|1450|
|  1|14.39|1.87|2.45|14.6| 96| 2.5|2.52| 0.3|1.98|5.25|1.02|3.58|1290|
|  1|14.06|2.15|2.61|17.6|121| 2.6|2.51|0.31|1.25|5.05|1.06|3.58|1295|
|  1|14.83|1.64|2.17|14.0| 97| 2.8|2.98|0.29|1.98| 5.2|1.08|2.85|1045|
|  1|13.86|1.35|2.27|16.0| 98|2.98|3.15|0.22|1.85|7.22|1.01|3.55|1045|
|  1| 14.1|2.16| 2.3|18.0|105|2.95|3.32|0.22|2.38|5.75|1.25|3.17|1510|
|  1|1

In [27]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+-----------------+-------------------+------------------+-----------------+
|summary|               _c0|               _c1|               _c2|               _c3|              _c4|               _c5|               _c6|               _c7|                _c8|               _c9|             _c10|               _c11|              _c12|             _c13|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+-----------------+-------------------+------------------+-----------------+
|  count|               178|               178|               178|               178|              178|               178|               178|               178|               

In [28]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=df.columns[1:], outputCol="features")
data_2 = assembler.transform(df)

In [29]:
data_2.show()

+---+-----+----+----+----+---+----+----+----+----+----+----+----+----+--------------------+
|_c0|  _c1| _c2| _c3| _c4|_c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|            features|
+---+-----+----+----+----+---+----+----+----+----+----+----+----+----+--------------------+
|  1|14.23|1.71|2.43|15.6|127| 2.8|3.06|0.28|2.29|5.64|1.04|3.92|1065|[14.23,1.71,2.43,...|
|  1| 13.2|1.78|2.14|11.2|100|2.65|2.76|0.26|1.28|4.38|1.05| 3.4|1050|[13.2,1.78,2.14,1...|
|  1|13.16|2.36|2.67|18.6|101| 2.8|3.24| 0.3|2.81|5.68|1.03|3.17|1185|[13.16,2.36,2.67,...|
|  1|14.37|1.95| 2.5|16.8|113|3.85|3.49|0.24|2.18| 7.8|0.86|3.45|1480|[14.37,1.95,2.5,1...|
|  1|13.24|2.59|2.87|21.0|118| 2.8|2.69|0.39|1.82|4.32|1.04|2.93| 735|[13.24,2.59,2.87,...|
|  1| 14.2|1.76|2.45|15.2|112|3.27|3.39|0.34|1.97|6.75|1.05|2.85|1450|[14.2,1.76,2.45,1...|
|  1|14.39|1.87|2.45|14.6| 96| 2.5|2.52| 0.3|1.98|5.25|1.02|3.58|1290|[14.39,1.87,2.45,...|
|  1|14.06|2.15|2.61|17.6|121| 2.6|2.51|0.31|1.25|5.05|1.06|3.58|1295|[14.06,2.1

In [30]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

In [31]:
scaler_model = scaler.fit(data_2)

In [32]:
scaled_data = scaler_model.transform(data_2)

In [33]:
scaled_data.show()

+---+-----+----+----+----+---+----+----+----+----+----+----+----+----+--------------------+--------------------+
|_c0|  _c1| _c2| _c3| _c4|_c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|            features|     scaled_features|
+---+-----+----+----+----+---+----+----+----+----+----+----+----+----+--------------------+--------------------+
|  1|14.23|1.71|2.43|15.6|127| 2.8|3.06|0.28|2.29|5.64|1.04|3.92|1065|[14.23,1.71,2.43,...|[17.5283750084766...|
|  1| 13.2|1.78|2.14|11.2|100|2.65|2.76|0.26|1.28|4.38|1.05| 3.4|1050|[13.2,1.78,2.14,1...|[16.2596310690015...|
|  1|13.16|2.36|2.67|18.6|101| 2.8|3.24| 0.3|2.81|5.68|1.03|3.17|1185|[13.16,2.36,2.67,...|[16.2103594597015...|
|  1|14.37|1.95| 2.5|16.8|113|3.85|3.49|0.24|2.18| 7.8|0.86|3.45|1480|[14.37,1.95,2.5,1...|[17.7008256410266...|
|  1|13.24|2.59|2.87|21.0|118| 2.8|2.69|0.39|1.82|4.32|1.04|2.93| 735|[13.24,2.59,2.87,...|[16.3089026783015...|
|  1| 14.2|1.76|2.45|15.2|112|3.27|3.39|0.34|1.97|6.75|1.05|2.85|1450|[14.2,1.76,2.45,1...|[17.4

In [34]:
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler(min=0, max=1, inputCol='features', outputCol='features_minmax')
scaler_model = scaler.fit(data_2)
data_3 = scaler_model.transform(data_2)

In [35]:
data_3.show()

+---+-----+----+----+----+---+----+----+----+----+----+----+----+----+--------------------+--------------------+
|_c0|  _c1| _c2| _c3| _c4|_c5| _c6| _c7| _c8| _c9|_c10|_c11|_c12|_c13|            features|     features_minmax|
+---+-----+----+----+----+---+----+----+----+----+----+----+----+----+--------------------+--------------------+
|  1|14.23|1.71|2.43|15.6|127| 2.8|3.06|0.28|2.29|5.64|1.04|3.92|1065|[14.23,1.71,2.43,...|[0.84210526315789...|
|  1| 13.2|1.78|2.14|11.2|100|2.65|2.76|0.26|1.28|4.38|1.05| 3.4|1050|[13.2,1.78,2.14,1...|[0.57105263157894...|
|  1|13.16|2.36|2.67|18.6|101| 2.8|3.24| 0.3|2.81|5.68|1.03|3.17|1185|[13.16,2.36,2.67,...|[0.56052631578947...|
|  1|14.37|1.95| 2.5|16.8|113|3.85|3.49|0.24|2.18| 7.8|0.86|3.45|1480|[14.37,1.95,2.5,1...|[0.87894736842105...|
|  1|13.24|2.59|2.87|21.0|118| 2.8|2.69|0.39|1.82|4.32|1.04|2.93| 735|[13.24,2.59,2.87,...|[0.58157894736842...|
|  1| 14.2|1.76|2.45|15.2|112|3.27|3.39|0.34|1.97|6.75|1.05|2.85|1450|[14.2,1.76,2.45,1...|[0.83

# Principal component analysis/dimension reduction

In [36]:
file_loc = 'gs://wac-buck/digits.csv'
df = spark.read.csv(file_loc,header=True,
                    inferSchema=True)

In [39]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=df.columns[1:], outputCol='features')
data_2 = assembler.transform(df)

In [41]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

In [42]:
pca = PCA(k=2, inputCol='features', outputCol='features_pca')

In [43]:
pca_model = pca.fit(data_2)

In [44]:
pca_data = pca_model.transform(data_2).select('features_pca')

In [45]:
pca_data.show()

+--------------------+
|        features_pca|
+--------------------+
|[103.738813757988...|
|[2466.78627830941...|
|[-121.55984060477...|
|[599.578991071954...|
|[2689.04430947599...|
|[1253.08650413365...|
|[93.0114290617972...|
|[650.952778816166...|
|[1115.56395904828...|
|[1062.72668192117...|
|[1029.01690081557...|
|[458.805321389773...|
|[-200.34133976161...|
|[751.263926957190...|
|[1265.44211418056...|
|[-199.11010313255...|
|[762.715694923051...|
|[1744.79986516160...|
|[128.314928856545...|
|[1731.44148649030...|
+--------------------+
only showing top 20 rows



# Titanic classification example

In [46]:
file_loc = 'gs://wac-buck/titanic.csv'
df = spark.read.csv(file_loc,header=True,
                    inferSchema=True)

In [47]:
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|Gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [48]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [49]:
df.describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|Gender|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

* It does not make sense to include some features such as: PassengerID, Name and Ticket → we will drop them
* Cabin has a lot of null values → we will drop it as well
* Maybe the Embarked column has nothing to do with the survival → let us remove it
* We are missing 177 values from the Age column → Age is important, we need to find a way to deal with the missing values
* Gender has nominal values → need to encode them

In [51]:
data = df.select(['Survived', 'Pclass', 'Gender', 'Age', 'SibSp', 'Parch', 'Fare'])

# Impute values - change nulls to means

There are many available strategies, but we will follow a simple one that fills missing values with the mean value calculated from the sample.

MLlib makes the job easy using the Imputer class. First, we define the estimator, fit it to the model, then we apply the transformer on the data

In [52]:
from pyspark.ml.feature import Imputer
imputer = Imputer(strategy='mean', inputCols=['Age'], outputCols=['AgeImputed'])
imputer_model = imputer.fit(data)
data = imputer_model.transform(data)

In [53]:
data.show()

+--------+------+------+----+-----+-----+-------+-----------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|       AgeImputed|
+--------+------+------+----+-----+-----+-------+-----------------+
|       0|     3|  male|22.0|    1|    0|   7.25|             22.0|
|       1|     1|female|38.0|    1|    0|71.2833|             38.0|
|       1|     3|female|26.0|    0|    0|  7.925|             26.0|
|       1|     1|female|35.0|    1|    0|   53.1|             35.0|
|       0|     3|  male|35.0|    0|    0|   8.05|             35.0|
|       0|     3|  male|null|    0|    0| 8.4583|29.69911764705882|
|       0|     1|  male|54.0|    0|    0|51.8625|             54.0|
|       0|     3|  male| 2.0|    3|    1| 21.075|              2.0|
|       1|     3|female|27.0|    0|    2|11.1333|             27.0|
|       1|     2|female|14.0|    1|    0|30.0708|             14.0|
|       1|     3|female| 4.0|    1|    1|   16.7|              4.0|
|       1|     1|female|58.0|    0|    0|  26.55

In [54]:
from pyspark.ml.feature import StringIndexer
gender_indexer = StringIndexer(inputCol='Gender', outputCol='GenderIndexed')
gender_indexer_model = gender_indexer.fit(data)
data = gender_indexer_model.transform(data)

In [55]:
data.show()

+--------+------+------+----+-----+-----+-------+-----------------+-------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|       AgeImputed|GenderIndexed|
+--------+------+------+----+-----+-----+-------+-----------------+-------------+
|       0|     3|  male|22.0|    1|    0|   7.25|             22.0|          0.0|
|       1|     1|female|38.0|    1|    0|71.2833|             38.0|          1.0|
|       1|     3|female|26.0|    0|    0|  7.925|             26.0|          1.0|
|       1|     1|female|35.0|    1|    0|   53.1|             35.0|          1.0|
|       0|     3|  male|35.0|    0|    0|   8.05|             35.0|          0.0|
|       0|     3|  male|null|    0|    0| 8.4583|29.69911764705882|          0.0|
|       0|     1|  male|54.0|    0|    0|51.8625|             54.0|          0.0|
|       0|     3|  male| 2.0|    3|    1| 21.075|              2.0|          0.0|
|       1|     3|female|27.0|    0|    2|11.1333|             27.0|          1.0|
|       1|     2

In [56]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Pclass', 'SibSp', 'Parch', 'Fare', 'AgeImputed', 'GenderIndexed'], outputCol='features')
data = assembler.transform(data)

In [57]:
data.show()

+--------+------+------+----+-----+-----+-------+-----------------+-------------+--------------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|       AgeImputed|GenderIndexed|            features|
+--------+------+------+----+-----+-----+-------+-----------------+-------------+--------------------+
|       0|     3|  male|22.0|    1|    0|   7.25|             22.0|          0.0|[3.0,1.0,0.0,7.25...|
|       1|     1|female|38.0|    1|    0|71.2833|             38.0|          1.0|[1.0,1.0,0.0,71.2...|
|       1|     3|female|26.0|    0|    0|  7.925|             26.0|          1.0|[3.0,0.0,0.0,7.92...|
|       1|     1|female|35.0|    1|    0|   53.1|             35.0|          1.0|[1.0,1.0,0.0,53.1...|
|       0|     3|  male|35.0|    0|    0|   8.05|             35.0|          0.0|[3.0,0.0,0.0,8.05...|
|       0|     3|  male|null|    0|    0| 8.4583|29.69911764705882|          0.0|[3.0,0.0,0.0,8.45...|
|       0|     1|  male|54.0|    0|    0|51.8625|             54.0|      

In [58]:
from pyspark.ml.classification import RandomForestClassifier
algo = RandomForestClassifier(featuresCol='features', labelCol='Survived')
model = algo.fit(data)

In [59]:
predictions = model.transform(data)

In [60]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(labelCol='Survived', metricName='areaUnderROC')

In [61]:
evaluator.evaluate(predictions)

0.8998737736874062

In [62]:
y_true = predictions.select(['Survived']).collect()
y_pred = predictions.select(['prediction']).collect()

In [63]:
from sklearn.metrics import classification_report, confusion_matrix

In [64]:
print(classification_report(y_true, y_pred))

ValueError: all the input arrays must have same number of dimensions

In [65]:
print(confusion_matrix(y_true, y_pred))

[[498  51]
 [ 92 250]]
